In [1]:
import qctoolkit as qtk
import numpy as np
import glob
import os, re
import copy

/home/samio/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cyl_map = {
    'si': [
        ['si', 5.431, [14, 14]],
        ['c', 3.566, [6, 6]],
        ['sic', 4.3596, [14, 6]],
        ['cge', 5.574, [6, 32]],
        ['ge', 5.658, [32, 32]],
        ['sige', 5.432, [14, 32]],
        ['sn', 6.4892, [50, 50]],
        ['snsi', 5.961, [14, 50]],
        ['gesn', 6.0758, [32, 50]],
    ],
    'gaas': [
        ['gaas', 5.6535, [31, 33]],
        ['alp', 5.4635, [13, 15]],
        ['alas', 5.660, [13, 33]],
        ['alsb', 6.1355, [13, 51]],
        ['gap', 5.451, [31, 15]],
        ['gasb', 6.09, [31, 51]],
        ['inp', 5.86, [49, 15]],
        ['inas', 6.05, [49, 33]],
        ['insb', 6.47, [49, 51]],
    ],
    'cdse': [
        ['cdse', 6.08, [48, 34]],
        ['zns', 5.41, [30, 16]],
        ['znse', 5.67, [30, 34]],
        ['znte', 6.1, [30, 52]],
        ['cds', 5.82, [48,16]],
        ['cdte', 6.48, [48, 52]],
        ['hgs', 5.8517, [80, 16]],
        ['hgse', 6.085, [80, 34]],
        ['hgte', 6.453, [80, 52]],
    ]
}

# plain GGA, where insuite restart is possible

In [3]:
# single setting dict for insuit restart calculations
qmsetting = {
    'program': 'abinit',
    'kmesh': [6,6,6],
    'ks_states': 10,
    'band_scan': [
     [17, 17, 8, 10, 15],
     [[0.25, 0.75, 0.5], # W
      [0.0, 0.0, 0.0],   # Gamma
      [0.0, 0.5, 0.5],   # X
      [0.25, 0.75, 0.5], # L
      [0.5, 0.5, 0.5],   # W
      [0.0, 0.0, 0.0],   # Gamma
     ]],
    'save_restart': True,
    'save_density': True,
    'link_dep': True,
    'threads': 1,
    'dos_mesh': [12, 12, 12],
    'smearing': 'gaussian',
}

In [4]:
# construct crystals based on cyl_map and si, gaas, cdse crystals

mol_base = []
#for cyl in ['si', 'gaas', 'cdse']:
for cyl in ['si', 'gaas']:
    name = 'xyz/%s.xyz' % cyl
    mol_base.append(qtk.Molecule(name))
mol_base

mols_grp = []
for mol in mol_base:
    for info_mol in cyl_map[mol.name]:
        mol_grp = []
        for info_a in cyl_map[mol.name]:
            new = mol.copy()
            new.name = info_mol[0] + '_a-' + info_a[0]
            new.setCelldm([info_a[1], info_a[1], info_a[1], 0, 0, 0])
            for i in range(new.N):
                if new.Z[i] != info_mol[2][i]:
                    new.setAtoms(i, Z=info_mol[2][i])
            mol_grp.append(new)
        mols_grp.append(mol_grp)
    
mols_all = list(qtk.flatten(mols_grp))
inps_ref = [qtk.QMInp(mol, **qmsetting) for mol in mols_all]

In [5]:
inps_ref

[si_a-si: abinit,
 si_a-c: abinit,
 si_a-sic: abinit,
 si_a-cge: abinit,
 si_a-ge: abinit,
 si_a-sige: abinit,
 si_a-sn: abinit,
 si_a-snsi: abinit,
 si_a-gesn: abinit,
 c_a-si: abinit,
 c_a-c: abinit,
 c_a-sic: abinit,
 c_a-cge: abinit,
 c_a-ge: abinit,
 c_a-sige: abinit,
 c_a-sn: abinit,
 c_a-snsi: abinit,
 c_a-gesn: abinit,
 sic_a-si: abinit,
 sic_a-c: abinit,
 sic_a-sic: abinit,
 sic_a-cge: abinit,
 sic_a-ge: abinit,
 sic_a-sige: abinit,
 sic_a-sn: abinit,
 sic_a-snsi: abinit,
 sic_a-gesn: abinit,
 cge_a-si: abinit,
 cge_a-c: abinit,
 cge_a-sic: abinit,
 cge_a-cge: abinit,
 cge_a-ge: abinit,
 cge_a-sige: abinit,
 cge_a-sn: abinit,
 cge_a-snsi: abinit,
 cge_a-gesn: abinit,
 ge_a-si: abinit,
 ge_a-c: abinit,
 ge_a-sic: abinit,
 ge_a-cge: abinit,
 ge_a-ge: abinit,
 ge_a-sige: abinit,
 ge_a-sn: abinit,
 ge_a-snsi: abinit,
 ge_a-gesn: abinit,
 sige_a-si: abinit,
 sige_a-c: abinit,
 sige_a-sic: abinit,
 sige_a-cge: abinit,
 sige_a-ge: abinit,
 sige_a-sige: abinit,
 sige_a-sn: abinit,
 si

# HSE06, fresh restart is necessary

In [8]:
qmsetting_base = {
    'cutoff': 10,
    'program': 'abinit',
    'save_restart': True,
    'save_density': True, 
    'link_dep': True, 
    'threads': 1,
    'ks_states': 10,
}

q_bnd = {
    'restart': True,
    'restart_density': True,
    'band_scan': [
     [15, 20],
     [[0.5, 0.0, 0.25],  # L
      [0.0, 0.0, 0.0],   # Gamma
      [0.0, 0.5, 0.5],   # X
     ]],
}
qmsetting_bnd = copy.deepcopy(qmsetting_base)
qmsetting_bnd.update(q_bnd)

q_dos = {
    'restart': True,
    'restart_density': True,
    'dos_mesh': [12,12,12],
}
qmsetting_dos = copy.deepcopy(qmsetting_base)
qmsetting_dos.update(q_dos)

ref_den_path = 'Eg_ref_vary_a_den_rst'
ref_bnd_path = 'Eg_ref_vary_a_bnd_rst'
ref_dos_path = 'Eg_ref_vary_a_dos_rst'

In [14]:
mol_base = []
#for cyl in ['si', 'gaas', 'cdse']:
for cyl in ['si', 'gaas']:
    name = 'xyz/%s.xyz' % cyl
    mol_base.append(qtk.Molecule(name))
mol_base

mols_grp = []
for mol in mol_base:
    for info_mol in cyl_map[mol.name]:
        mol_grp = []
        for info_a in cyl_map[mol.name]:
            new = mol.copy()
            new.name = info_mol[0] + '_a-' + info_a[0]
            new.setCelldm([info_a[1], info_a[1], info_a[1], 0, 0, 0])
            for i in range(new.N):
                if new.Z[i] != info_mol[2][i]:
                    new.setAtoms(i, Z=info_mol[2][i])
            mol_grp.append(new)
        mols_grp.append(mol_grp)
    
mols_all = list(qtk.flatten(mols_grp))
mols_all.reverse()

inps_ref_den = [qtk.QMInp(mol, kmesh = [5,5,5], **qmsetting_base) for mol in mols_all]

cwd = os.getcwd() + '/'
inps_ref_bnd = []
inps_ref_dos = []
for mol in mols_all:
    inp_bnd = qtk.QMInp(mol, 
                        restart_density_path = cwd + ref_den_path + '/%s' % mol.name,
                        **qmsetting_bnd)
    inp_dos = qtk.QMInp(mol, 
                        restart_density_path = cwd + ref_den_path + '/%s' % mol.name,
                        **qmsetting_dos)
    inps_ref_bnd.append(inp_bnd)
    inps_ref_dos.append(inp_dos)
    

In [16]:
len(inps_ref_bnd)

162

In [18]:
inps_ref_bnd[0].write()

 pp file In_pbe_pbe not found in /home/samio/.local/lib/python2.7/site-packages/qctoolkit-0.0.14-py2.7-linux-x86_64.egg/qctoolkit/data/PP/bigdft. But found in cp2k page, download now... 
 pp file Sb_pbe_pbe not found in /home/samio/.local/lib/python2.7/site-packages/qctoolkit-0.0.14-py2.7-linux-x86_64.egg/qctoolkit/data/PP/bigdft. But found in cp2k page, download now... 
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref_vary_a_den_rst/insb_a-insb not found
# abinit input generated by qctoolkit

# datasets section
ndtset 2

# scf section
ngkpt1 5 5 5
shiftk1 0.0 0.0 0.0

# band_scan section
iscf2 -2
getden2 -1
kptopt2 -2
tolwfr2 1e-05
ndivk2 15 20
kptbounds2 0.5 0.0 0.25
           0.0 0.0 0.0
           0.0 0.5 0.5
prtwf2 1
prtden2 1

# cell section
ecut 5.0
nstep 1000
acell 1 1 1
rprim 0.0 6.11326401435 6.11326401435
      6.11326401435 0.0 6.11326401435
      6.11326401435 6.11326401435 0.0

# atoms section
natom 2
ntypat 2
typat 1 2
znucl 49 51
xred

     0.0 0.0 0.0
